In [ ]:
import cv2
import numpy as np
import insightface
from insightface.app import FaceAnalysis
import faiss
import os
import pickle
from collections import Counter
from IPython.display import display, Image, clear_output
import ipywidgets as widgets

In [ ]:
MODEL_NAME = 'buffalo_l'
DB_INDEX_FILE = 'face_index.faiss'
LABELS_FILE = 'labels.pkl'
EMBEDDING_SIZE = 512
MAX_FACES = 4

face_app = FaceAnalysis(name=MODEL_NAME)
face_app.prepare(ctx_id=0, det_size=(640, 640))

if os.path.exists(DB_INDEX_FILE):
    face_index = faiss.read_index(DB_INDEX_FILE)
    with open(LABELS_FILE, 'rb') as f:
        face_labels = pickle.load(f)
else:
    face_index = faiss.IndexFlatL2(EMBEDDING_SIZE)
    face_labels = []

In [ ]:
def process_image(img):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    faces = face_app.get(img_rgb)
    
    if len(faces) == 0:
        return None, "Лица не обнаружены"
    
    if len(faces) > MAX_FACES:
        return None, f"Слишком много лиц на фото ({len(faces)}). Максимально допустимо: {MAX_FACES}."
    
    main_face = sorted(faces, key=lambda x: (x.bbox[2]-x.bbox[0])*(x.bbox[3]-x.bbox[1]), reverse=True)[0]
    return main_face.normed_embedding, None

def show_database_stats():
    if face_labels:
        print("\n📊 Статистика базы данных:")
        counter = Counter(face_labels)
        for name, count in counter.items():
            print(f" - {name}: {count} {'лицо' if count == 1 else 'лица'}")
        print(f"Всего лиц: {len(face_labels)}")
    else:
        print("База данных пуста")

def save_database():
    faiss.write_index(face_index, DB_INDEX_FILE)
    with open(LABELS_FILE, 'wb') as f:
        pickle.dump(face_labels, f)
    print("✅ База данных сохранена")

In [ ]:
def add_face(uploader):
    if not uploader.value:
        print("⚠️ Сначала загрузите изображение")
        return
    
    person_name = name_input.value.strip()
    if not person_name:
        print("⚠️ Введите имя человека")
        return
    
    for name, data in uploader.value.items():
        img_data = data['content']
        img = cv2.imdecode(np.frombuffer(img_data, np.uint8), cv2.IMREAD_COLOR)
        
        display(Image(data=img_data, width=300))
        
        embedding, error = process_image(img)
        if error:
            print(f"❌ Ошибка: {error}")
            return
        
        if face_labels:
            distances, _ = face_index.search(np.array([embedding]), 1)
            if distances[0][0] < 0.5:
                print("⚠️ Это лицо уже существует в базе данных!")
                return
        
        face_index.add(np.array([embedding]))
        face_labels.append(person_name)
        save_database()
        print(f"✅ Лицо '{person_name}' успешно добавлено!")
        show_database_stats()
        break

def search_face(uploader):
    if not uploader.value:
        print("⚠️ Сначала загрузите изображение для поиска")
        return
    
    for name, data in uploader.value.items():
        img_data = data['content']
        img = cv2.imdecode(np.frombuffer(img_data, np.uint8), cv2.IMREAD_COLOR)
        
        display(Image(data=img_data, width=300))
        
        embedding, error = process_image(img)
        if error:
            print(f"❌ Ошибка: {error}")
            return
        
        if not face_labels:
            print("❌ База данных пуста!")
            return
            
        distances, indices = face_index.search(np.array([embedding]), 3)
        
        found = False
        for i in range(len(distances[0])):
            if distances[0][i] < 0.6:
                match = face_labels[indices[0][i]]
                confidence = max(0, 100 - distances[0][i] * 100)
                print(f"✅ Найдено совпадение: {match} (Уверенность: {confidence:.1f}%)")
                found = True
                break
        
        if not found:
            print("❌ Совпадений в базе не найдено")
        
        if face_labels:
            print("\n🔍 Топ-3 возможных совпадения:")
            for i in range(len(distances[0])):
                idx = indices[0][i]
                confidence = max(0, 100 - distances[0][i] * 100)
                print(f"{i+1}. {face_labels[idx]} ({confidence:.1f}%)")

In [ ]:
print("➕ ДОБАВЛЕНИЕ НОВОГО ЛИЦА В БАЗУ")
add_uploader = widgets.FileUpload(accept='image/*', multiple=False)
display(add_uploader)

name_input = widgets.Text(placeholder="Введите имя (например: Моя мама)")
display(name_input)

add_button = widgets.Button(description="Добавить в базу")
add_button.on_click(lambda b: add_face(add_uploader))
display(add_button)

print("\n" + "="*50 + "\n")

print("🔍 ПОИСК ЛИЦ В БАЗЕ ДАННЫХ")
search_uploader = widgets.FileUpload(accept='image/*', multiple=False)
display(search_uploader)

search_button = widgets.Button(description="Найти в базе")
search_button.on_click(lambda b: search_face(search_uploader))
display(search_button)

show_database_stats()